### Python libraries import

In [159]:
import csv
import pandas as pd
from pyDOE2 import *
import numpy as np

### Cell-free buffer parameters

In [171]:
# Open CSV input and parse for parameters
# file = open('proCFPS_parameters.csv')
# csvreader = csv.reader(file)
# parameters = []
# for parameter in csvreader:
#         parameters.append(parameter)
# file.close()
# parameters

# # Open CSV input and parse for parameters
# file = open('proCFPS_parameters.csv')
# csvreader = csv.reader(file)
# parameters = {}
# for parameter in csvreader:
#         parameters = {key: row for key, *row in csvreader}
# file.close()

# parameters.values()
# variable_parameters = []
# fixed_parameters = []
# maximum_concentration = []
# parameters['Status'] = parameters['First_name'].apply(lambda x: 'Found' if x == 'Ria' else 'Not Found')

parameters = pd.read_csv('proCFPS_parameters.csv')
concentrations = parameters[['Parameter', 'Maximum concentration']]
concentrations = concentrations.to_dict()
concentrations

{'Parameter': {0: 'Mg-glutamate',
  1: 'k-glutamate',
  2: 'Sodium oxalate',
  3: '3-PGA',
  4: 'NTP',
  5: 'NAD',
  6: 'Folinic acid',
  7: 'Spermidine',
  8: 'tRNA',
  9: 'Amino acids',
  10: 'Extract',
  11: 'HEPES',
  12: 'GOI-DNA',
  13: 'GFP-DNA'},
 'Maximum concentration': {0: 10.4,
  1: 355.0,
  2: 2.1,
  3: 30.0,
  4: 1.5,
  5: 30.0,
  6: 0.2,
  7: 1.3,
  8: 2.0,
  9: 0.5,
  10: 10.0,
  11: 30.0,
  12: 50.0,
  13: 50.0}}

### Latin hypercube sampling

In [142]:
# Define number of parameters
n_parameters = len(parameters)
# Replace with n_parameters = len(variable_parameters)

# Define number of samples to generate
n_samples = 100

# Define number of ratios
n_ratios = 5

# Perform latin hypercube sampling
sampling = lhs(n_parameters, samples=n_samples)

In [143]:
# Define ratios
levels = (1 / n_ratios)

level_1 = levels
level_2 = level_1*2
level_3 = level_1*3
level_4 = level_1*4
level_5 = level_1*5

# Generate multi-level list
levels_list = []

for experiment in sampling:
    new_experiment = []
    
    for parameter in experiment:
        
        if parameter < level_1:
            new_experiment.append(0)
            continue
        
        if parameter > level_1 and parameter <= level_2:
            new_experiment.append(level_1)
            continue
        
        if parameter > level_2 and parameter <= level_3:
            new_experiment.append(level_2)
            continue
        
        if parameter > level_3 and parameter <= level_4:
            new_experiment.append(level_3)
            continue
            
        if parameter > level_4 and parameter <= level_5:
            new_experiment.append(level_4)
            continue
    
    levels_list.append(new_experiment)

# Convert multi-level list to a multi-level array
levels_array = np.asarray(levels_list)

### Generating multi-level concentration array

In [148]:
# Define maximum concentrations for each cell-free buffer parameter
maximum_concentrations_array = np.array([10.4, 355, 2.1, 30, 1.5, 30, 0.2, 1.3, 2, 0.5, 10, 30])

# Multiply ratios array by maximum concentrations vector
variable_concentrations_array = np.multiply(levels_array, maximum_concentrations_array)
fixed_concentrations_array = (np.full((100, 1), 50))
final_concentration_array = np.concatenate(variable_concentrations_array, fixed_concentrations_array, axis=None)

TypeError: concatenate() got multiple values for argument 'axis'

In [133]:
# Save output in a csv file
headers = "Mg-glutamate; k-glutamate; Sodium oxalate; 3-PGA; NTP; NAD; Folinic acid; Spermidine; tRNA; Amino acids; Extract; HEPES; GOI-DNA; GFP-DNA"
output = np.savetxt("initial_plate.csv", concentrations_array, delimiter=";", header = headers, fmt='%.5f')